In [1]:
import gzip
import csv
import json
import math
import random
import tensorflow as tf
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [2]:
def readData(fp, split):
    """
    Train/val data reader that have different headers for pos/neg entries.
    
    split: number of positive/negative items (should be total // 2)
    """
    result = []
    with open(fp, 'r') as f:
        header = next(f).strip().split(',')
        non_ints = {'stream', 'streamer'}
        i = 0
        for line in f:
            # parse data
            content = line.strip().split(',')
            if i < split:
                d = dict(zip(header, content))
                for k, v in d.items():
                    if not k in non_ints:
                        d[k] = int(v)
            else:
                d = {
                    'user': int(content[0]),
                    'stream': content[1],
                    'watched': int(content[-1])
                }    
            i += 1
            result.append(d)
            
    return result

def readCSV(fp):
    """
    Data reader for regular CSV files, returns a list of dictionaries.
    """
    result = []
    with open(fp, 'r') as f:
        header = next(f).strip().split(',')
        for line in f:
            d = dict(zip(header, line.strip().split(',')))
            result.append(d)
    return result

def readX(fp):
    """
    Data reader for processed feature vector files, returns a 2D-list where
    each element is a list of feature vectors.
    """
    result = []
    with open(fp, 'r') as f:
        header = next(f)
        print("Header: " + header.strip())
        for line in f:
            l = line.strip().split(',')
            result.append(l)
    return result

def readJSON(fp):
    """
    JSON data reader.
    """
    with open(fp, 'r') as f:
        return json.load(f)

In [3]:
# read from file
train = readData('data/train.txt', split=12350800)
val = readData('data/val.txt', split=2446606)
train_streams = readJSON('data/train_streams.json')
val_streams = readJSON('data/val_streams.json')

# get labels
train_y = [d['watched'] for d in train]
val_y = [d['watched'] for d in val]

In [87]:
def accuracy(pred, y):
    return np.mean(np.array(pred) == np.array(y))

### FPMC

In [4]:
userIDs = {}
streamerIDs = {}
interactions = []
interactionsPerUser = defaultdict(list)

for d in train[:12350800]:
    u = int(d['user'])
    i = train_streams[d['stream']]['streamer']
    t = int(d['time_stop'])
    w = int(d['watched'])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in streamerIDs: streamerIDs[i] = len(streamerIDs)
    interactions.append((t,u,i,w))
    interactionsPerUser[u].append((t,i,w))

In [5]:
interactions.sort()

In [6]:
streamerIDs['CSE158DUMMYFIRSTITEM'] = len(streamerIDs)

In [7]:
interactionsWithPrevious = []

for u in interactionsPerUser:
    interactionsPerUser[u].sort()
    lastItem = 'CSE158DUMMYFIRSTITEM'
    for (t,i,w) in interactionsPerUser[u]:
        interactionsWithPrevious.append((t,u,i,lastItem,w))
        lastItem = i

In [8]:
itemsPerUser = defaultdict(set)
for _,u,i,_ in interactions:
    itemsPerUser[u].add(i)

In [9]:
items = list(streamerIDs.keys())

#### TensorFlow Model

In [10]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [11]:
class FPMC(tf.keras.Model):
    def __init__(self, K, lamb, UI = 1, IJ = 1):
        super(FPMC, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(streamerIDs)],stddev=0.001))
        self.gammaUI = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaIU = tf.Variable(tf.random.normal([len(streamerIDs),K],stddev=0.001))
        self.gammaIJ = tf.Variable(tf.random.normal([len(streamerIDs),K],stddev=0.001))
        self.gammaJI = tf.Variable(tf.random.normal([len(streamerIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb
        # Which terms to include
        self.UI = UI
        self.IJ = IJ

    # Prediction for a single instance
    def predict(self, u, i, j):
        if not i in self.betaI:
            return 0
        if not j in self.gammaJI:
            return self.betaI[i] + self.UI * tf.tensordot(self.gammaUI[u], self.gammaIU[i], 1)
        
        return self.betaI[i] + self.UI * tf.tensordot(self.gammaUI[u], self.gammaIU[i], 1) +\
                            self.IJ * tf.tensordot(self.gammaIJ[i], self.gammaJI[j], 1)

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaUI) +\
                            tf.nn.l2_loss(self.gammaIU) +\
                            tf.nn.l2_loss(self.gammaIJ) +\
                            tf.nn.l2_loss(self.gammaJI))

    def call(self, sampleU, # user
                   sampleI, # item
                   sampleJ, # previous item
                   sampleK): # negative item
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        j = tf.convert_to_tensor(sampleJ, dtype=tf.int32)
        k = tf.convert_to_tensor(sampleK, dtype=tf.int32)
        gamma_ui = tf.nn.embedding_lookup(self.gammaUI, u)
        gamma_iu = tf.nn.embedding_lookup(self.gammaIU, i)
        gamma_ij = tf.nn.embedding_lookup(self.gammaIJ, i)
        gamma_ji = tf.nn.embedding_lookup(self.gammaJI, j)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        x_uij = beta_i + self.UI * tf.reduce_sum(tf.multiply(gamma_ui, gamma_iu), 1) +\
                         self.IJ * tf.reduce_sum(tf.multiply(gamma_ij, gamma_ji), 1)
        gamma_uk = tf.nn.embedding_lookup(self.gammaUI, u)
        gamma_ku = tf.nn.embedding_lookup(self.gammaIU, k)
        gamma_kj = tf.nn.embedding_lookup(self.gammaIJ, k)
        gamma_jk = tf.nn.embedding_lookup(self.gammaJI, j)
        beta_k = tf.nn.embedding_lookup(self.betaI, k)
        x_ukj = beta_k + self.UI * tf.reduce_sum(tf.multiply(gamma_uk, gamma_ku), 1) +\
                         self.IJ * tf.reduce_sum(tf.multiply(gamma_kj, gamma_jk), 1)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_uij - x_ukj)))

In [12]:
modelFPMC = FPMC(5, 0.00001)

In [13]:
def trainingStep(model, interactions):
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ, sampleK = [], [], [], []
        for _ in range(500000):
            _,u,i,j,_ = random.choice(interactions) # positive sample
            k = random.choice(items) # negative sample
            while k in itemsPerUser[u]:
                k = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(streamerIDs[i])
            sampleJ.append(streamerIDs[j])
            sampleK.append(streamerIDs[k])

        loss = model(sampleU,sampleI,sampleJ,sampleK)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [14]:
i = 0
prev_obj = float('inf')
inc = 0
while i <= 100:
    obj = trainingStep(modelFPMC, interactionsWithPrevious)
    print("iteration " + str(i+1) + ", objective = " + str(obj))
    if obj >= prev_obj: 
        inc += 1
    else:
        inc = 0
    if inc >= 3: break
    prev_obj = obj
    i += 1

iteration 1, objective = 0.69320697
iteration 2, objective = 0.6483494
iteration 3, objective = 0.607886
iteration 4, objective = 0.5721485
iteration 5, objective = 0.5393749
iteration 6, objective = 0.5073709
iteration 7, objective = 0.4741845
iteration 8, objective = 0.44022834
iteration 9, objective = 0.40666625
iteration 10, objective = 0.37583184
iteration 11, objective = 0.34955913
iteration 12, objective = 0.3294412
iteration 13, objective = 0.31715715
iteration 14, objective = 0.31026852
iteration 15, objective = 0.30682185
iteration 16, objective = 0.3069491
iteration 17, objective = 0.3081665
iteration 18, objective = 0.31052265


In [41]:
def fpmc_predict(model, u, i, j):
    assert isinstance(model, FPMC)
    if i < 0:
        return 0
    if j < 0:
        return model.betaI[i] + model.UI * tf.tensordot(model.gammaUI[u], model.gammaIU[i], 1)

    return model.betaI[i] + model.UI * tf.tensordot(model.gammaUI[u], model.gammaIU[i], 1) +\
                        model.IJ * tf.tensordot(model.gammaIJ[i], model.gammaJI[j], 1)

In [19]:
actionsPerUser = defaultdict(list)

for idx, d in enumerate(train):
    u = d['user']
    i = train_streams[d['stream']]['streamer']
    t = train_streams[d['stream']]['time_start']
    w = d['watched']
    actionsPerUser[u].append((t,i,w,idx))

In [20]:
trainWithPrevious = []

for u in actionsPerUser:
    actionsPerUser[u].sort()
    lastItem = 'CSE158DUMMYFIRSTITEM'
    for (t,i,w,idx) in actionsPerUser[u]:
        trainWithPrevious.append((t,u,i,lastItem,w, idx))
        if w == 1:
            lastItem = i

In [21]:
trainWithPrevious.sort(key=lambda t:t[-1])

In [24]:
def find_param_fpmc_train(t):
    _, user, cur, prev, _, _ = t
    user_id = userIDs[user]
    if cur in streamerIDs:
        cur_item = streamerIDs[cur]
    else:
        cur_item = -1
    if prev in streamerIDs:
        prev_item = streamerIDs[prev]
    else:
        prev_item = -1
    return fpmc_predict(modelFPMC, user_id, cur_item, prev_item)

In [40]:
def find_param_fpmc_val(*args):
    user, cur, prev = args
    if user in userIDs:
        user_id = userIDs[user]
    else:
        user_id = -1
    if cur in streamerIDs:
        cur_item = streamerIDs[cur]
    else:
        cur_item = -1
    if prev in streamerIDs:
        prev_item = streamerIDs[prev]
    else:
        prev_item = -1
    return fpmc_predict(modelFPMC, user_id, cur_item, prev_item)

In [17]:
# for each user find last train item to be used as prev_item in validation
last_train = {u: lst[-1][1] for u, lst in interactionsPerUser.items()}

In [47]:
streams_per_user_val = defaultdict(list)

for i, d in enumerate(val):
    if (i + 1) % 1000 == 0: print(f'Progress: {i} / 4893212')
    user = d['user']
    cur = val_streams[d['stream']]['streamer']
    prev = last_train[user] if user in last_train else 'CSE158DUMMYFIRSTITEM'
    
    streams_per_user_val[u].append((find_param_fpmc_val(user, cur, prev), i))

Progress: 999 / 4893212
Progress: 1999 / 4893212
Progress: 2999 / 4893212
Progress: 3999 / 4893212
Progress: 4999 / 4893212
Progress: 5999 / 4893212
Progress: 6999 / 4893212
Progress: 7999 / 4893212
Progress: 8999 / 4893212
Progress: 9999 / 4893212
Progress: 10999 / 4893212
Progress: 11999 / 4893212
Progress: 12999 / 4893212
Progress: 13999 / 4893212
Progress: 14999 / 4893212
Progress: 15999 / 4893212
Progress: 16999 / 4893212
Progress: 17999 / 4893212
Progress: 18999 / 4893212
Progress: 19999 / 4893212
Progress: 20999 / 4893212
Progress: 21999 / 4893212
Progress: 22999 / 4893212
Progress: 23999 / 4893212
Progress: 24999 / 4893212
Progress: 25999 / 4893212
Progress: 26999 / 4893212
Progress: 27999 / 4893212
Progress: 28999 / 4893212
Progress: 29999 / 4893212
Progress: 30999 / 4893212
Progress: 31999 / 4893212
Progress: 32999 / 4893212
Progress: 33999 / 4893212
Progress: 34999 / 4893212
Progress: 35999 / 4893212
Progress: 36999 / 4893212
Progress: 37999 / 4893212
Progress: 38999 / 48932

Progress: 308999 / 4893212
Progress: 309999 / 4893212
Progress: 310999 / 4893212
Progress: 311999 / 4893212
Progress: 312999 / 4893212
Progress: 313999 / 4893212
Progress: 314999 / 4893212
Progress: 315999 / 4893212
Progress: 316999 / 4893212
Progress: 317999 / 4893212
Progress: 318999 / 4893212
Progress: 319999 / 4893212
Progress: 320999 / 4893212
Progress: 321999 / 4893212
Progress: 322999 / 4893212
Progress: 323999 / 4893212
Progress: 324999 / 4893212
Progress: 325999 / 4893212
Progress: 326999 / 4893212
Progress: 327999 / 4893212
Progress: 328999 / 4893212
Progress: 329999 / 4893212
Progress: 330999 / 4893212
Progress: 331999 / 4893212
Progress: 332999 / 4893212
Progress: 333999 / 4893212
Progress: 334999 / 4893212
Progress: 335999 / 4893212
Progress: 336999 / 4893212
Progress: 337999 / 4893212
Progress: 338999 / 4893212
Progress: 339999 / 4893212
Progress: 340999 / 4893212
Progress: 341999 / 4893212
Progress: 342999 / 4893212
Progress: 343999 / 4893212
Progress: 344999 / 4893212
P

Progress: 612999 / 4893212
Progress: 613999 / 4893212
Progress: 614999 / 4893212
Progress: 615999 / 4893212
Progress: 616999 / 4893212
Progress: 617999 / 4893212
Progress: 618999 / 4893212
Progress: 619999 / 4893212
Progress: 620999 / 4893212
Progress: 621999 / 4893212
Progress: 622999 / 4893212
Progress: 623999 / 4893212
Progress: 624999 / 4893212
Progress: 625999 / 4893212
Progress: 626999 / 4893212
Progress: 627999 / 4893212
Progress: 628999 / 4893212
Progress: 629999 / 4893212
Progress: 630999 / 4893212
Progress: 631999 / 4893212
Progress: 632999 / 4893212
Progress: 633999 / 4893212
Progress: 634999 / 4893212
Progress: 635999 / 4893212
Progress: 636999 / 4893212
Progress: 637999 / 4893212
Progress: 638999 / 4893212
Progress: 639999 / 4893212
Progress: 640999 / 4893212
Progress: 641999 / 4893212
Progress: 642999 / 4893212
Progress: 643999 / 4893212
Progress: 644999 / 4893212
Progress: 645999 / 4893212
Progress: 646999 / 4893212
Progress: 647999 / 4893212
Progress: 648999 / 4893212
P

Progress: 916999 / 4893212
Progress: 917999 / 4893212
Progress: 918999 / 4893212
Progress: 919999 / 4893212
Progress: 920999 / 4893212
Progress: 921999 / 4893212
Progress: 922999 / 4893212
Progress: 923999 / 4893212
Progress: 924999 / 4893212
Progress: 925999 / 4893212
Progress: 926999 / 4893212
Progress: 927999 / 4893212
Progress: 928999 / 4893212
Progress: 929999 / 4893212
Progress: 930999 / 4893212
Progress: 931999 / 4893212
Progress: 932999 / 4893212
Progress: 933999 / 4893212
Progress: 934999 / 4893212
Progress: 935999 / 4893212
Progress: 936999 / 4893212
Progress: 937999 / 4893212
Progress: 938999 / 4893212
Progress: 939999 / 4893212
Progress: 940999 / 4893212
Progress: 941999 / 4893212
Progress: 942999 / 4893212
Progress: 943999 / 4893212
Progress: 944999 / 4893212
Progress: 945999 / 4893212
Progress: 946999 / 4893212
Progress: 947999 / 4893212
Progress: 948999 / 4893212
Progress: 949999 / 4893212
Progress: 950999 / 4893212
Progress: 951999 / 4893212
Progress: 952999 / 4893212
P

Progress: 1212999 / 4893212
Progress: 1213999 / 4893212
Progress: 1214999 / 4893212
Progress: 1215999 / 4893212
Progress: 1216999 / 4893212
Progress: 1217999 / 4893212
Progress: 1218999 / 4893212
Progress: 1219999 / 4893212
Progress: 1220999 / 4893212
Progress: 1221999 / 4893212
Progress: 1222999 / 4893212
Progress: 1223999 / 4893212
Progress: 1224999 / 4893212
Progress: 1225999 / 4893212
Progress: 1226999 / 4893212
Progress: 1227999 / 4893212
Progress: 1228999 / 4893212
Progress: 1229999 / 4893212
Progress: 1230999 / 4893212
Progress: 1231999 / 4893212
Progress: 1232999 / 4893212
Progress: 1233999 / 4893212
Progress: 1234999 / 4893212
Progress: 1235999 / 4893212
Progress: 1236999 / 4893212
Progress: 1237999 / 4893212
Progress: 1238999 / 4893212
Progress: 1239999 / 4893212
Progress: 1240999 / 4893212
Progress: 1241999 / 4893212
Progress: 1242999 / 4893212
Progress: 1243999 / 4893212
Progress: 1244999 / 4893212
Progress: 1245999 / 4893212
Progress: 1246999 / 4893212
Progress: 1247999 / 

...

Progress: 3556999 / 4893212
Progress: 3557999 / 4893212
Progress: 3558999 / 4893212
Progress: 3559999 / 4893212
Progress: 3560999 / 4893212
Progress: 3561999 / 4893212
Progress: 3562999 / 4893212
Progress: 3563999 / 4893212
Progress: 3564999 / 4893212
Progress: 3565999 / 4893212
Progress: 3566999 / 4893212
Progress: 3567999 / 4893212
Progress: 3568999 / 4893212
Progress: 3569999 / 4893212
Progress: 3570999 / 4893212
Progress: 3571999 / 4893212
Progress: 3572999 / 4893212
Progress: 3573999 / 4893212
Progress: 3574999 / 4893212
Progress: 3575999 / 4893212
Progress: 3576999 / 4893212
Progress: 3577999 / 4893212
Progress: 3578999 / 4893212
Progress: 3579999 / 4893212
Progress: 3580999 / 4893212
Progress: 3581999 / 4893212
Progress: 3582999 / 4893212
Progress: 3583999 / 4893212
Progress: 3584999 / 4893212
Progress: 3585999 / 4893212
Progress: 3586999 / 4893212
Progress: 3587999 / 4893212
Progress: 3588999 / 4893212
Progress: 3589999 / 4893212
Progress: 3590999 / 4893212
Progress: 3591999 / 

Progress: 3849999 / 4893212
Progress: 3850999 / 4893212
Progress: 3851999 / 4893212
Progress: 3852999 / 4893212
Progress: 3853999 / 4893212
Progress: 3854999 / 4893212
Progress: 3855999 / 4893212
Progress: 3856999 / 4893212
Progress: 3857999 / 4893212
Progress: 3858999 / 4893212
Progress: 3859999 / 4893212
Progress: 3860999 / 4893212
Progress: 3861999 / 4893212
Progress: 3862999 / 4893212
Progress: 3863999 / 4893212
Progress: 3864999 / 4893212
Progress: 3865999 / 4893212
Progress: 3866999 / 4893212
Progress: 3867999 / 4893212
Progress: 3868999 / 4893212
Progress: 3869999 / 4893212
Progress: 3870999 / 4893212
Progress: 3871999 / 4893212
Progress: 3872999 / 4893212
Progress: 3873999 / 4893212
Progress: 3874999 / 4893212
Progress: 3875999 / 4893212
Progress: 3876999 / 4893212
Progress: 3877999 / 4893212
Progress: 3878999 / 4893212
Progress: 3879999 / 4893212
Progress: 3880999 / 4893212
Progress: 3881999 / 4893212
Progress: 3882999 / 4893212
Progress: 3883999 / 4893212
Progress: 3884999 / 

Progress: 4142999 / 4893212
Progress: 4143999 / 4893212
Progress: 4144999 / 4893212
Progress: 4145999 / 4893212
Progress: 4146999 / 4893212
Progress: 4147999 / 4893212
Progress: 4148999 / 4893212
Progress: 4149999 / 4893212
Progress: 4150999 / 4893212
Progress: 4151999 / 4893212
Progress: 4152999 / 4893212
Progress: 4153999 / 4893212
Progress: 4154999 / 4893212
Progress: 4155999 / 4893212
Progress: 4156999 / 4893212
Progress: 4157999 / 4893212
Progress: 4158999 / 4893212
Progress: 4159999 / 4893212
Progress: 4160999 / 4893212
Progress: 4161999 / 4893212
Progress: 4162999 / 4893212
Progress: 4163999 / 4893212
Progress: 4164999 / 4893212
Progress: 4165999 / 4893212
Progress: 4166999 / 4893212
Progress: 4167999 / 4893212
Progress: 4168999 / 4893212
Progress: 4169999 / 4893212
Progress: 4170999 / 4893212
Progress: 4171999 / 4893212
Progress: 4172999 / 4893212
Progress: 4173999 / 4893212
Progress: 4174999 / 4893212
Progress: 4175999 / 4893212
Progress: 4176999 / 4893212
Progress: 4177999 / 

Progress: 4435999 / 4893212
Progress: 4436999 / 4893212
Progress: 4437999 / 4893212
Progress: 4438999 / 4893212
Progress: 4439999 / 4893212
Progress: 4440999 / 4893212
Progress: 4441999 / 4893212
Progress: 4442999 / 4893212
Progress: 4443999 / 4893212
Progress: 4444999 / 4893212
Progress: 4445999 / 4893212
Progress: 4446999 / 4893212
Progress: 4447999 / 4893212
Progress: 4448999 / 4893212
Progress: 4449999 / 4893212
Progress: 4450999 / 4893212
Progress: 4451999 / 4893212
Progress: 4452999 / 4893212
Progress: 4453999 / 4893212
Progress: 4454999 / 4893212
Progress: 4455999 / 4893212
Progress: 4456999 / 4893212
Progress: 4457999 / 4893212
Progress: 4458999 / 4893212
Progress: 4459999 / 4893212
Progress: 4460999 / 4893212
Progress: 4461999 / 4893212
Progress: 4462999 / 4893212
Progress: 4463999 / 4893212
Progress: 4464999 / 4893212
Progress: 4465999 / 4893212
Progress: 4466999 / 4893212
Progress: 4467999 / 4893212
Progress: 4468999 / 4893212
Progress: 4469999 / 4893212
Progress: 4470999 / 

Progress: 4728999 / 4893212
Progress: 4729999 / 4893212
Progress: 4730999 / 4893212
Progress: 4731999 / 4893212
Progress: 4732999 / 4893212
Progress: 4733999 / 4893212
Progress: 4734999 / 4893212
Progress: 4735999 / 4893212
Progress: 4736999 / 4893212
Progress: 4737999 / 4893212
Progress: 4738999 / 4893212
Progress: 4739999 / 4893212
Progress: 4740999 / 4893212
Progress: 4741999 / 4893212
Progress: 4742999 / 4893212
Progress: 4743999 / 4893212
Progress: 4744999 / 4893212
Progress: 4745999 / 4893212
Progress: 4746999 / 4893212
Progress: 4747999 / 4893212
Progress: 4748999 / 4893212
Progress: 4749999 / 4893212
Progress: 4750999 / 4893212
Progress: 4751999 / 4893212
Progress: 4752999 / 4893212
Progress: 4753999 / 4893212
Progress: 4754999 / 4893212
Progress: 4755999 / 4893212
Progress: 4756999 / 4893212
Progress: 4757999 / 4893212
Progress: 4758999 / 4893212
Progress: 4759999 / 4893212
Progress: 4760999 / 4893212
Progress: 4761999 / 4893212
Progress: 4762999 / 4893212
Progress: 4763999 / 

In [49]:
val[0]

{'user': 1,
 'stream': '34329206336',
 'streamer': 'yoda',
 'time_start': 5205,
 'time_stop': 5206,
 'time_start_in_day': 21,
 'time_start_at_day': 36,
 'time_stop_in_day': 22,
 'time_stop_at_day': 36,
 'watched': 1}

In [51]:
dd = val[0]
user = dd['user']
cur = val_streams[dd['stream']]['streamer']
prev = last_train[user]
ts = find_param_fpmc_val(user, cur, prev)

In [53]:
ts.numpy()

6.1732016

In [58]:
streams_per_user_val[500000][0]

(<tf.Tensor: shape=(), dtype=float32, numpy=6.1732016>, 0)

In [61]:
per_user_val = defaultdict(list)

for tup in tqdm(streams_per_user_val[500000]):
    idx = tup[-1]
    score = tup[0]
    if not isinstance(score, int):
        score = score.numpy()
    user = val[idx]['user']
    per_user_val[user].append((score, idx))

  0%|          | 0/4893212 [00:00<?, ?it/s]

In [77]:
fpmc_pred_dict = dict()

for u in per_user_val:
    lst = per_user_val[u]
    lst.sort(reverse=True)
    full_size = len(lst)
    pos_size = full_size // 2
    for i in range(full_size):
        if i < pos_size:
            fpmc_pred_dict[lst[i][-1]] = 1
        else:
            fpmc_pred_dict[lst[i][-1]] = 0

In [84]:
fpmc_pred = [fpmc_pred_dict[i] for i in range(4893212)]

In [86]:
with open('data/fpmcPredictions.txt', 'w') as f:
    f.write('prediction\n')
    for pred in fpmc_pred:
        f.write(str(pred) + '\n')

In [89]:
accuracy(fpmc_pred, val_y)

0.8368903697612121